# DNN 프로젝트 느낀 점
: 따라하다 보니 완성이었다...

- 앞에 배운 것들을 제대로 복습하고 넘어가야 할 듯(언제...?)
- 재밌어!
- 딥러닝은 천재야

## 1. 데이터 불러오기

In [4]:
import torch
import torch.nn as nn
import torch.optim
import torchvision.datasets as dset
import torchvision.transforms as transforms

In [3]:
#하이퍼 파라미터
batch_size = 100
num_epochs = 5
learning_rate = 0.001

In [ ]:
# DataLoader

from torch.utils.data import DataLoader

root = './data'
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
train_data = dset.FashionMNIST(root=root, train=True, transform=transform, download=True)
test_data = dset.FashionMNIST(root=root, train=False, transform=transform, download=True)
## 코드 시작 ##
train_loader = DataLoader(dataset=train_data, batch_size = batch_size, shuffle = True, drop_last = True)
test_loader = DataLoader(dataset=test_data, batch_size = batch_size, shuffle = False, drop_last = True)
## 코드 종료 ##

## 2. 네트워크 설계
: Multi Layer Perceptron 레이어 2개!

In [ ]:
class DNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DNN, self).__init__()
        self.layer1 = nn.Sequential(
            ## 코드 시작 ##
            torch.nn.Linear(784, 512, bias=True),    # Linear_1 해당하는 층
            torch.nn.BatchNorm1d(512),    # BatchNorm_1 해당하는 층
            torch.nn.ReLU()    # ReLU_1 해당하는 층
            ## 코드 종료 ##
        )
        self.layer2 = nn.Sequential(
            ## 코드 시작 ##
            torch.nn.Linear(512, 10, bias = True)    # Linear_2 해당하는 층 
            ## 코드 종료 ##
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.layer1(x)
        x_out = self.layer2(x_out)
        return x_out

## Batch Normalization 주의 !!!
# Batch Normalization에 해당하는 값은 출력값에 해당! = 512
# 순서: 입력 -> 정규화 -> active function -> 입력 -> softmax (crossEntropy에 구현)

## 3. model 생성 및 cost 계산 및 개선

In [ ]:
#모델 생성

def weights_init(m): # 가중치 초기화
    if isinstance(m, nn.Linear): # 모델의 모든 MLP 레이어에 대해서
        nn.init.xavier_normal_(m.weight) # Weight를 xavier_normal로 초기화
        print(m.weight)

torch.manual_seed(7777) # 일관된 weight initialization을 위한 random seed 설정
model = DNN()
model.apply(weights_init) # 모델에 weight_init 함수를 적용하여 weight를 초기화

## 코드 시작 ##
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
## 코드 종료 ##

## 4. Training

In [ ]:
for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs, labels
        ## 코드 시작 ##
        outputs = model(imgs) #y값
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step()
        ## 코드 종료 ##
        
        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                epoch+1, num_epochs, i+1, len(train_loader), loss.item(), accuracy.item() * 100))

## 5. Training과 Test
- model.eval(): 모델을 **평가모드**로 설정. -> batch normalization과 dropout이 training을 할 때와 test를 할 때 작동하는 방식이 다름
- model.train(): 모델을 **훈련모드**로 설정